In [26]:
import glob, os, json, re, unicodedata
from bs4 import BeautifulSoup


# load JSON text files; parsing into raw text
directory = "../data/unprocessed/"
text_dir = "../data/processed3/"

count = 0
for filename in glob.glob(os.path.join(directory, '*.json')):
        json_data=open(filename)
        data = json.load(json_data)
        print filename
        for data2 in data["speeches"]:
            speaker = data2['speaker']

            if ('News Conference With' not in data2['name']) and ('News Conference with' not in data2['name']) and (' With President ' not in data2['name']):
                count += 1

                if (speaker == "Barack Obama"):
                    print speaker, data2['name'], data2['date']

                name = speaker.split(' ')
                
                if ('W. Bush' in speaker):
                    pres_lastname = 'WBush'
                else:
                    pres_lastname = name[len(name) - 1]

                outfile = format('%s_NewsConf%s_%s.txt' % (pres_lastname, count, data2['date']))
                # Remove ' ' and comma
                outfile = re.sub('[\s,\,]','', outfile, flags=re.DOTALL)

                txtfile = open(text_dir + outfile, 'w')
                # data2['text'] has a lot of htmtl tags in there. We still need to parse it            
                raw = BeautifulSoup(data2['text'], "html.parser").get_text(" ")
                raw = unicodedata.normalize('NFKD', raw).encode('ascii','ignore')
                # Remove []
                raw = re.sub(' \[.*?\]',' ', raw, flags=re.DOTALL)
                # Remove ()
                raw = re.sub(' \(.*?\)',' ', raw, flags=re.DOTALL)
                # Removing the questions
                raw = re.sub('[A-Z,\s,\.]Q\..*? The President\.','\.',raw, flags=re.DOTALL)
                raw = re.sub('^[A-Z,\s]*THE PRESIDENT\.','',raw, flags=re.DOTALL)
                raw = re.sub('[A-Z,\s,\.]Q\..*?THE PRESIDENT\.','\.',raw, flags=re.DOTALL)
                
#                print "writing data to", outfile
                txtfile.write(raw)
                txtfile.close()
                
        json_data.close()
print "Count =", count

                

../data/unprocessed/Presidential_News_Confs_(1929-2017).json
Barack Obama The President's News Conference February 9, 2009
Barack Obama The President's News Conference March 24, 2009
Barack Obama The President's News Conference in London April 2, 2009
Barack Obama The President's News Conference in Strasbourg April 4, 2009
Barack Obama The President's News Conference in Port of Spain April 19, 2009
Barack Obama The President's News Conference April 29, 2009
Barack Obama The President's News Conference June 23, 2009
Barack Obama The President's News Conference in L'Aquila, Italy July 10, 2009
Barack Obama The President's News Conference July 22, 2009
Barack Obama The President's News Conference in Pittsburgh September 25, 2009
Barack Obama The President's News Conference in Copenhagen December 18, 2009
Barack Obama The President's News Conference February 9, 2010
Barack Obama The President's News Conference April 13, 2010
Barack Obama The President's News Conference May 27, 2010
Barack 

### After that, we need to do some more clean up from the unix environment

grep -l "^ Citation:" *txt | awk '{print "sed -i -e \"\" \"/^ Citation/d\" " $1 }' > x

./x

Remove "NOTE: ....":  grep -l " NOTE: " *txt | awk '{print "sed \"s/ NOTE:.*//\" " $1 " > y; mv y " $1}' > x

Replace "\." with newline: grep -l " \\\. " *txt | awk '{print "sed  \"s/\\\\\\./\\'\\\n'/g\" " $1 " > x; mv x " $1}' > z

grep -l "^The President. " *txt > x
cat x | awk '{print "sed \"s/^The President.//g\" " $1 " > y; mv y " $1}' > z

Remove "--": grep -l "\-\- \-\-" *txt | awk '{print "sed  \"s/\\-\\- \\-\\-//g\" " $1 " > x; mv x " $1}' > z
